# <center>IEE 520: Fall 2019</center>

# <center> Cluster Analysis (11/21/19)</center>

## <center>Klim Drobnyh (klim.drobnyh@asu.edu)</center>
### <center>Special thanks to Maziar Kasaei for suggestions</center>

**NOTE: TO SUPPORT INTERACTIVE PLOTS IN JUPYTER LAB, RUN**

conda install -c conda-forge nodejs

jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
# For compatibility with Python 2
from __future__ import print_function

# To load datasets
from sklearn import datasets

# To import clustering-related methods
from scipy.cluster import hierarchy
from sklearn.cluster import KMeans

import seaborn as sns; sns.set()

from sklearn.preprocessing import MinMaxScaler

# Dimensionality reduction
from sklearn.decomposition import PCA

# To measure accuracy
from sklearn import metrics

# To support plots
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
import pandas as pd

import math

# To display all the plots inline
%matplotlib inline

In [ ]:
# To increase quality of figures
plt.rcParams["figure.figsize"] = (20, 7)

## <center>K-Means</center>

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

### <center>Digits dataset</center>

Dataset: https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where each class refers to a digit.

Preprocessing programs made available by NIST were used to extract normalized bitmaps of handwritten digits from a preprinted form. From a total of 43 people, 30 contributed to the training set and different 13 to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of 4x4 and the number of on pixels are counted in each block. This generates an input matrix of 8x8 where each element is an integer in the range 0..16. This reduces dimensionality and gives invariance to small distortions.

In [ ]:
X, y = datasets.load_digits(n_class=10, return_X_y=True)

Now, let's use PCA (principal component analysis) to reduce dimension to 2, so we can simply visualize the data.

In [ ]:
pca_model = PCA(n_components=2)
X_reduced = pca_model.fit_transform(X)

In [ ]:
# Here we use closure to store the related variables
def create_plot_kmeans_digits(_X, _y, _n_classes):
    X, y = _X, _y
    n_classes = _n_classes
    colors = np.array([cm.tab20(i) for i in range(100)])
    def plot_kmeans_digits(n_clusters=10):
        expand=1
        kmeans = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10, random_state=520, max_iter=1000)
        kmeans.fit(X)
        y_predicted = kmeans.predict(X)
        fig, (ax1, ax2) = plt.subplots(1, 2)
        ax1.plot((np.min(X_reduced[:, 0])-expand, np.max(X_reduced[:, 0])+expand), 
                 (np.min(X_reduced[:, 1])-expand, np.max(X_reduced[:, 1])+expand),
                 alpha=0.0)
        xlim = ax1.get_xlim()
        ylim = ax1.get_ylim()
        ax1.scatter(X_reduced[:, 0], X_reduced[:, 1], c=colors[y_predicted])
        ax1.set_xlabel('pca #1')
        ax1.set_ylabel('pca #2') 
        centers = np.array(kmeans.cluster_centers_)
        ax1.scatter(centers[:, 0], centers[:, 1], marker="x", color='k', s=64)
        matrix = np.zeros((n_clusters, n_classes))
        for cluster in range(n_clusters):
            for cl in range(n_classes):
                matrix[cluster, cl] = np.sum(y[y_predicted == cluster]==cl)
        sns.heatmap(matrix, annot=True, fmt='g', ax=ax2)
        ax2.set_xlabel('Class')
        ax2.set_ylabel('Cluster')
        print('Inertia:', kmeans.inertia_)
        print('Homogenity:', metrics.homogeneity_score(y, y_predicted))
        print('Completeness:', metrics.completeness_score(y, y_predicted))
        plt.show()
    return plot_kmeans_digits

In [ ]:
n_clusters_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    step=1,
    continuous_update=False,
    description='N clusters:')
interact(create_plot_kmeans_digits(X_reduced, y, 10),
         n_clusters=n_clusters_widget)

There are multiple ways of cluster evaluation. 
- Class labels are not available: within-cluster sum of squares (inertia) can be used.
- Class labels are available: measures like Completeness and Homogeneity can be used. 

In [ ]:
n_clusters = range(1, 30)
inertia = np.zeros(len(n_clusters))
homogeneity = np.zeros(len(n_clusters))
completeness = np.zeros(len(n_clusters))
for i, clusters in enumerate(n_clusters):
    kmeans = KMeans(init='k-means++', n_clusters=clusters, n_init=10, random_state=520, max_iter=1000)
    kmeans.fit(X)
    inertia[i] = kmeans.inertia_
    y_predicted = kmeans.predict(X)
    homogeneity[i] = metrics.homogeneity_score(y, y_predicted)
    completeness[i] = metrics.completeness_score(y, y_predicted)

In [ ]:
plt.plot(n_clusters, completeness)
plt.title('Kmeans: completeness')
plt.xlabel('# of clusters')
plt.ylabel('Completeness')
plt.show()

In [ ]:
plt.plot(n_clusters, homogeneity)
plt.title('Kmeans: homogeneity')
plt.xlabel('# of clusters')
plt.ylabel('Homogeneity')
plt.show()

In [ ]:
plt.plot(n_clusters, inertia)
plt.title('Kmeans: inertia')
plt.xlabel('# of clusters')
plt.ylabel('Inertia')
plt.show()

## <center>Hierarchical clustering</center>

https://scikit-learn.org/stable/modules/clustering.html#hierarchical-clustering

https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html

### <center>Parkinsons Data Set</center>

Parkinsons disease: https://www.mayoclinic.org/diseases-conditions/parkinsons-disease/symptoms-causes/syc-20376055

Dataset: https://archive.ics.uci.edu/ml/datasets/parkinsons

This dataset is composed of a range of biomedical voice measurements from 31 people, 23 with Parkinson's disease (PD). Each column in the table is a particular voice measure, and each row corresponds one of 195 voice recording from these individuals ("name" column). The main aim of the data is to discriminate healthy people from those with PD, according to "status" column which is set to 0 for healthy and 1 for PD.

* name - ASCII subject name and recording number
* MDVP:Fo(Hz) - Average vocal fundamental frequency
* MDVP:Fhi(Hz) - Maximum vocal fundamental frequency
* MDVP:Flo(Hz) - Minimum vocal fundamental frequency
* MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP - Several measures of variation in fundamental frequency
* MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA - Several measures of variation in amplitude
* NHR,HNR - Two measures of ratio of noise to tonal components in the voice
* status - Health status of the subject (one) - Parkinson's, (zero) - healthy
* RPDE,D2 - Two nonlinear dynamical complexity measures
* DFA - Signal fractal scaling exponent
* spread1,spread2,PPE - Three nonlinear measures of fundamental frequency variation

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data')
print(df.columns)
X = df.drop(['name','status'], axis=1).values
y = df['status'].values
print(X.shape)
print(y.shape)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# Here we use a closure to store the related variables
def create_plot_hierarchical(_X, _y):
    X, y = _X, _y
    n_classes = len(np.unique(y))
    def plot_hierarchical(threshold=1.0, method='single'):
        expand = 1
        linkage = hierarchy.linkage(X, method=method, metric='euclidean')
        
        fig, (ax1, ax2) = plt.subplots(2, 1)
        
        hierarchy.dendrogram(linkage,
                   labels=y,
                   leaf_rotation=0,
                   leaf_font_size=7,
                   color_threshold=threshold, 
                   ax=ax1)
        
        y_predicted = hierarchy.fcluster(linkage, t=threshold, criterion='distance')
        
        n_clusters = len(np.unique(y_predicted))
        
        matrix = np.zeros((n_clusters, n_classes))
        for cluster in range(n_clusters):
            for cl in range(n_classes):
                matrix[cluster, cl] = np.sum(y[y_predicted == cluster+1]==cl)
        sns.heatmap(matrix, annot=True, fmt='g', ax=ax2)
        ax2.set_xlabel('Class')
        ax2.set_ylabel('Cluster')
        print('Homogenity:', metrics.homogeneity_score(y, y_predicted))
        print('Completeness:', metrics.completeness_score(y, y_predicted))
        plt.show()
    return plot_hierarchical

In [ ]:
threshold_widget = widgets.FloatSlider(
    value=1,
    min=0.0,
    max=10.0,
    step=0.05,
    continuous_update=False,
    description='Threshold:')
methods = ['single', 'complete', 'average', 'ward']
interact(create_plot_hierarchical(X_scaled, y),
         threshold=threshold_widget, method=methods)

### <center>SPECT Heart Data Set</center>

Datset: https://archive.ics.uci.edu/ml/datasets/spect+heart

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal and abnormal. The database of 267 SPECT image sets (patients) was processed to extract features that summarize the original SPECT images. As a result, 44 continuous feature pattern was created for each patient. The pattern was further processed to obtain 22 binary feature patterns.

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECTF.train', header=None)
X = df.values[:, 1:]
y = df.values[:, 0]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
# Here we use a closure to store the related variables
def create_plot_hierarchical(_X, _y):
    X, y = _X, _y
    n_classes = len(np.unique(y))
    def plot_hierarchical(threshold=1.0, method='single'):
        expand = 1
        linkage = hierarchy.linkage(X, method=method, metric='euclidean')
        
        fig, (ax1, ax2) = plt.subplots(2, 1)
        
        hierarchy.dendrogram(linkage,
                   labels=y,
                   leaf_rotation=0,
                   leaf_font_size=7,
                   color_threshold=threshold, 
                   ax=ax1)
        
        y_predicted = hierarchy.fcluster(linkage, t=threshold, criterion='distance')
        
        n_clusters = len(np.unique(y_predicted))
        
        matrix = np.zeros((n_clusters, n_classes))
        for cluster in range(n_clusters):
            for cl in range(n_classes):
                matrix[cluster, cl] = np.sum(y[y_predicted == cluster+1]==cl)
        sns.heatmap(matrix, annot=True, fmt='g', ax=ax2)
        ax2.set_xlabel('Class')
        ax2.set_ylabel('Cluster')
        print('Homogenity:', metrics.homogeneity_score(y, y_predicted))
        print('Completeness:', metrics.completeness_score(y, y_predicted))
        plt.show()
    return plot_hierarchical

In [ ]:
threshold_widget = widgets.FloatSlider(
    value=1,
    min=0.0,
    max=10.0,
    step=0.05,
    continuous_update=False,
    description='Threshold:')
methods = ['single', 'complete', 'average', 'ward']
interact(create_plot_hierarchical(X_scaled, y),
         threshold=threshold_widget, method=methods)